## Regularization with SciKit-Learn
Previously we created a new polynomial feature set and then applied our standard linear regression on it, but we can be smarter about model choice and utilize regularization.

Regularization attempts to minimize the RSS (residual sum of squares) and a penalty factor. This penalty factor will penalize models that have coefficients that are too large. Some methods of regularization will actually cause non useful features to have a coefficient of zero, in which case the model does not consider the feature.

Let's explore two methods of regularization, Ridge Regression and Lasso. We'll combine these with the polynomial feature set (it wouldn't be as effective to perform regularization of a model on such a small original feature set of the original X).

In [1]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots

In [2]:
df = pd.read_csv('..\data\Advertising.csv')

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


## Data Setup

In [4]:
X = df.drop('sales',axis=1)
y = df['sales']

## Polynomial

In [5]:
from sklearn.preprocessing import PolynomialFeatures

In [6]:
poly_converter = PolynomialFeatures(degree= 3, include_bias = False)

In [7]:
poly_features = poly_converter.fit_transform(X)

## Train | Test Split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size= 0.3, random_state = 101)

## Scaling the data

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()

In [12]:
X_train = scaler.fit_transform(X_train)

In [13]:
X_test = scaler.transform(X_test)

In [14]:
X_train

array([[ 0.49300171, -0.33994238,  1.61586707, ..., -0.16360242,
         0.54694754,  1.37075536],
       [ 1.61337153,  1.32852213,  1.90079242, ...,  2.63236858,
         2.6297449 ,  1.95593378],
       [-0.84811893, -1.58789957, -1.02319564, ..., -0.72634944,
        -0.61593941, -0.54133745],
       ...,
       [ 1.04606563, -1.30982215,  0.27064572, ..., -0.71065643,
        -0.53420112, -0.20690952],
       [ 0.74817069,  0.03987068, -1.26608283, ..., -0.66805936,
        -0.61031703, -0.54616941],
       [ 0.13813882,  0.55533126,  1.01799092, ...,  0.61006767,
         0.67881488,  0.45425942]])

In [15]:
X_test

array([[-0.91339472,  1.76259419,  0.68168558, ...,  2.09516506,
         0.89465642,  0.09868885],
       [ 1.23121149,  0.23655959, -1.19601922, ..., -0.62566104,
        -0.60402516, -0.54547726],
       [-0.46002473, -0.40776614,  0.35005115, ..., -0.43068575,
        -0.23635113, -0.15776394],
       ...,
       [-1.33115973,  1.19965698, -1.18200649, ..., -0.47847952,
        -0.59569585, -0.54528297],
       [ 1.3522684 , -0.23142436, -1.4389064 , ..., -0.7233135 ,
        -0.61591305, -0.54660193],
       [ 0.27818504, -0.06186496,  0.01841672, ..., -0.32290124,
        -0.28905624, -0.33528306]])

## Rigde Regulation (L2)

In [16]:
from sklearn.linear_model import Ridge

In [17]:
help(Ridge)

Help on class Ridge in module sklearn.linear_model._ridge:

class Ridge(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, _BaseRidge)
 |  Ridge(alpha=1.0, *, fit_intercept=True, normalize='deprecated', copy_X=True, max_iter=None, tol=0.001, solver='auto', positive=False, random_state=None)
 |  
 |  Linear least squares with l2 regularization.
 |  
 |  Minimizes the objective function::
 |  
 |  ||y - Xw||^2_2 + alpha * ||w||^2_2
 |  
 |  This model solves a regression model where the loss function is
 |  the linear least squares function and regularization is given by
 |  the l2-norm. Also known as Ridge Regression or Tikhonov regularization.
 |  This estimator has built-in support for multi-variate regression
 |  (i.e., when y is a 2d-array of shape (n_samples, n_targets)).
 |  
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |  
 |  Parameters
 |  ----------
 |  alpha : {float, ndarray of shape (n_targets,)}, default=1.0
 |      Regularization strength; must be 

In [18]:
ridge_model = Ridge(alpha =10)

In [19]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [20]:
test_predictions = ridge_model.predict(X_test)

In [21]:
# evaluatian metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [22]:
MAE = mean_absolute_error(y_test, test_predictions)
MSE = mean_squared_error(y_test, test_predictions)
RMSE = np.sqrt(MSE)

In [23]:
MAE

0.5774404204714173

In [24]:
MSE

0.8003783071528403

In [25]:
RMSE

0.8946386461319678

How did it perform on the training set? (This will be used later on for comparison)

In [26]:
# training set performance
train_prediction = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train, train_prediction)
MAE

0.5288348183025328

## Choosing an aplha value with Cross_Validation

In [27]:
from sklearn.linear_model import RidgeCV

In [28]:
help(RidgeCV)

Help on class RidgeCV in module sklearn.linear_model._ridge:

class RidgeCV(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, _BaseRidgeCV)
 |  RidgeCV(alphas=(0.1, 1.0, 10.0), *, fit_intercept=True, normalize='deprecated', scoring=None, cv=None, gcv_mode=None, store_cv_values=False, alpha_per_target=False)
 |  
 |  Ridge regression with built-in cross-validation.
 |  
 |  See glossary entry for :term:`cross-validation estimator`.
 |  
 |  By default, it performs efficient Leave-One-Out Cross-Validation.
 |  
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |  
 |  Parameters
 |  ----------
 |  alphas : ndarray of shape (n_alphas,), default=(0.1, 1.0, 10.0)
 |      Array of alpha values to try.
 |      Regularization strength; must be a positive float. Regularization
 |      improves the conditioning of the problem and reduces the variance of
 |      the estimates. Larger values specify stronger regularization.
 |      Alpha corresponds to ``1 / (2C)`` in other lin

In [29]:
# Negative RMSE so all metrics follow convention "Higher is better"

# See all options: sklearn.metrics.SCORERS.keys()

ridgecv_model = RidgeCV(alphas = (0.1, 1.0, 10.0), scoring ='neg_mean_absolute_error')

In [30]:
ridgecv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [31]:
ridgecv_model.alpha_

0.1

In [32]:
test_predictions = ridgecv_model.predict(X_test)

In [33]:
# evaluation metrics
MAE = mean_absolute_error(y_test, test_predictions)
MSE = mean_squared_error(y_test, test_predictions)
RMSE = np.sqrt(MSE)

In [34]:
MAE

0.42737748843249934

In [35]:
RMSE

0.6180719926923581

In [36]:
# training set performance
train_predictions = ridgecv_model.predict(X_train)
MAE = mean_absolute_error(y_train, train_predictions)
MAE

0.3094132105641705

In [37]:
ridgecv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

## Lasso Regression (L1)

In [38]:
from sklearn.linear_model import LassoCV

In [39]:
help(LassoCV)

Help on class LassoCV in module sklearn.linear_model._coordinate_descent:

class LassoCV(sklearn.base.RegressorMixin, LinearModelCV)
 |  LassoCV(*, eps=0.001, n_alphas=100, alphas=None, fit_intercept=True, normalize='deprecated', precompute='auto', max_iter=1000, tol=0.0001, copy_X=True, cv=None, verbose=False, n_jobs=None, positive=False, random_state=None, selection='cyclic')
 |  
 |  Lasso linear model with iterative fitting along a regularization path.
 |  
 |  See glossary entry for :term:`cross-validation estimator`.
 |  
 |  The best model is selected by cross-validation.
 |  
 |  The optimization objective for Lasso is::
 |  
 |      (1 / (2 * n_samples)) * ||y - Xw||^2_2 + alpha * ||w||_1
 |  
 |  Read more in the :ref:`User Guide <lasso>`.
 |  
 |  Parameters
 |  ----------
 |  eps : float, default=1e-3
 |      Length of the path. ``eps=1e-3`` means that
 |      ``alpha_min / alpha_max = 1e-3``.
 |  
 |  n_alphas : int, default=100
 |      Number of alphas along the regulariz

In [40]:
lassocv_model = LassoCV(eps= 0.1, n_alphas = 100, cv=5)

In [41]:
lassocv_model.fit(X_train, y_train)

LassoCV(cv=5, eps=0.1)

In [42]:
test_predictions = lassocv_model.predict(X_test)

In [43]:
# evaluation metrics
MAE = mean_absolute_error(y_test, test_predictions)
MSE = mean_squared_error(y_test, test_predictions)
RMSE = np.sqrt(MSE)

In [44]:
MAE

0.6541723161252858

In [45]:
RMSE

1.1308001022762542

In [46]:
RMSE

1.1308001022762542

In [47]:
# training set performance
train_prediction = lassocv_model.predict(X_train)
MAE = mean_absolute_error(y_train, train_prediction)
MAE

0.69128071408207

In [48]:
lassocv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

## Elastic Net
Elastic Net combines the penalties of ridge regression and lasso in an attempt to get the best of both worlds!

In [49]:
from sklearn.linear_model import ElasticNetCV

In [50]:
help(ElasticNetCV)

Help on class ElasticNetCV in module sklearn.linear_model._coordinate_descent:

class ElasticNetCV(sklearn.base.RegressorMixin, LinearModelCV)
 |  ElasticNetCV(*, l1_ratio=0.5, eps=0.001, n_alphas=100, alphas=None, fit_intercept=True, normalize='deprecated', precompute='auto', max_iter=1000, tol=0.0001, cv=None, copy_X=True, verbose=0, n_jobs=None, positive=False, random_state=None, selection='cyclic')
 |  
 |  Elastic Net model with iterative fitting along a regularization path.
 |  
 |  See glossary entry for :term:`cross-validation estimator`.
 |  
 |  Read more in the :ref:`User Guide <elastic_net>`.
 |  
 |  Parameters
 |  ----------
 |  l1_ratio : float or list of float, default=0.5
 |      Float between 0 and 1 passed to ElasticNet (scaling between
 |      l1 and l2 penalties). For ``l1_ratio = 0``
 |      the penalty is an L2 penalty. For ``l1_ratio = 1`` it is an L1 penalty.
 |      For ``0 < l1_ratio < 1``, the penalty is a combination of L1 and L2
 |      This parameter can 

In [51]:
elastic_model = ElasticNetCV(l1_ratio= [.1, .5, .7,.9, .95, .99, 1], tol = 0.01)

In [52]:
elastic_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [53]:
elastic_model.l1_ratio_

1.0

In [54]:
test_predictions = elastic_model.predict(X_test)

In [55]:
# evaluation metrics
MAE = mean_absolute_error(y_test, test_predictions)
MSE = mean_squared_error(y_test, test_predictions)
RMSE = np.sqrt(MSE)

In [56]:
MAE

0.5663262117569451

In [57]:
MSE

0.5603340214638839

In [58]:
RMSE

0.7485546215633726

In [59]:
# training set peformance
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train, train_predictions)
MAE

0.4307582990472368

In [60]:
elastic_model.coef_

array([ 3.78993643,  0.89232919,  0.28765395, -1.01843566,  2.15516144,
       -0.3567547 , -0.271502  ,  0.09741081,  0.        , -1.05563151,
        0.2362506 ,  0.07980911,  1.26170778,  0.01464706,  0.00462336,
       -0.39986069,  0.        ,  0.        , -0.05343757])